In [1]:
import pandas as pd
import mplfinance as mpf
import numpy as np
from pandas_datareader import data as web
import yfinance as yf
yf.pdr_override()
import vectorbt as vbt
from datetime import datetime
from binance.client import Client
import os
import pandas_ta as ta

In [2]:
import sys
sys.path.append(r'C:\Users\gunsr\Desktop\Programming\Git Remote\Investic\Python-for-Investing-101\Global_Config')
import api_key

In [3]:
bn_key = api_key.binance_api_key
bn_secret = api_key.binance_api_secret

In [4]:
def get_data(ls_ticker, api_key, api_secret):
    
    client = Client(api_key, api_secret)
    
    ls_df = []
    
    for ticker in ls_ticker:
        klines = client.get_historical_klines(ticker, Client.KLINE_INTERVAL_1DAY, "1 Jan 2010")
        data = pd.DataFrame(klines)
        data.columns = ['open_time', 
                        'Open', 
                        'High', 
                        'Low',
                        'Close', 
                        'Volume',
                        'close_time',
                        'quote_asset_volume',
                        'number_of_trades',
                        'taker_buy_base_asset_volume',
                        'take_buy_quote_asset_volume',
                        'can_be_ignored']
        
        data['Date'] = data['open_time'].apply(lambda timestamp: datetime.fromtimestamp(int(str(timestamp)[:10])))
        df = data[['Date', 'Open', 'High', 'Low', 'Close', 'Volume']]
        df = df.astype({"Open": float, "High": float, "Low": float, "Close": float, "Volume": float})
        ls_df.append(df)
        #print(df.tail(10))
        
    return ls_df


In [5]:
ls_ticker = ['BTCUSDT', 'ETHUSDT']

data_dfs = get_data(ls_ticker, bn_key, bn_secret)

In [6]:
data_dfs[0]

,Date,Open,High,Low,Close,Volume
0,2017-08-17 07:00:00,4261.48,4485.39,4200.74,4285.08,795.150377
1,2017-08-18 07:00:00,4285.08,4371.52,3938.77,4108.37,1199.888264
2,2017-08-19 07:00:00,4108.37,4184.69,3850.00,4139.98,381.309763
3,2017-08-20 07:00:00,4120.98,4211.08,4032.62,4086.29,467.083022
4,2017-08-21 07:00:00,4069.13,4119.62,3911.79,4016.00,691.743060
...,...,...,...,...,...,...
1664,2022-03-08 07:00:00,37988.01,39362.08,37867.65,38730.63,55528.433670
1665,2022-03-09 07:00:00,38730.63,42594.06,38656.45,41941.71,67392.587990
1666,2022-03-10 07:00:00,41941.70,42039.63,38539.73,39422.00,71950.256770
1667,2022-03-11 07:00:00,39422.01,40236.26,38223.60,38729.57,59018.764200


In [7]:
df = data_dfs[0].copy()
df = df.set_index('Date')
df

,Open,High,Low,Close,Volume
Date,,,,,
2017-08-17 07:00:00,4261.48,4485.39,4200.74,4285.08,795.150377
2017-08-18 07:00:00,4285.08,4371.52,3938.77,4108.37,1199.888264
2017-08-19 07:00:00,4108.37,4184.69,3850.00,4139.98,381.309763
2017-08-20 07:00:00,4120.98,4211.08,4032.62,4086.29,467.083022
2017-08-21 07:00:00,4069.13,4119.62,3911.79,4016.00,691.743060
...,...,...,...,...,...
2022-03-08 07:00:00,37988.01,39362.08,37867.65,38730.63,55528.433670
2022-03-09 07:00:00,38730.63,42594.06,38656.45,41941.71,67392.587990
2022-03-10 07:00:00,41941.70,42039.63,38539.73,39422.00,71950.256770


In [8]:
df.ta.macd(12, 26, append=True)
df['signal'] = df.iloc[:,-3] > 0
df

,Open,High,Low,Close,Volume,MACD_12_26_9,MACDh_12_26_9,MACDs_12_26_9,signal
Date,,,,,,,,,
2017-08-17 07:00:00,4261.48,4485.39,4200.74,4285.08,795.150377,NaN,NaN,NaN,False
2017-08-18 07:00:00,4285.08,4371.52,3938.77,4108.37,1199.888264,NaN,NaN,NaN,False
2017-08-19 07:00:00,4108.37,4184.69,3850.00,4139.98,381.309763,NaN,NaN,NaN,False
2017-08-20 07:00:00,4120.98,4211.08,4032.62,4086.29,467.083022,NaN,NaN,NaN,False
2017-08-21 07:00:00,4069.13,4119.62,3911.79,4016.00,691.743060,NaN,NaN,NaN,False
...,...,...,...,...,...,...,...,...,...
2022-03-08 07:00:00,37988.01,39362.08,37867.65,38730.63,55528.433670,-422.621053,-211.755972,-210.865081,False
2022-03-09 07:00:00,38730.63,42594.06,38656.45,41941.71,67392.587990,-218.615938,-6.200686,-212.415252,False
2022-03-10 07:00:00,41941.70,42039.63,38539.73,39422.00,71950.256770,-257.294029,-35.903021,-221.391008,False


In [9]:
signal_vectorbt = df.ta.tsignals(df.signal, asbool=True, append=True)
df

,Open,High,Low,Close,Volume,MACD_12_26_9,MACDh_12_26_9,MACDs_12_26_9,signal,TS_Trends,TS_Trades,TS_Entries,TS_Exits
Date,,,,,,,,,,,,,
2017-08-17 07:00:00,4261.48,4485.39,4200.74,4285.08,795.150377,NaN,NaN,NaN,False,False,0,False,False
2017-08-18 07:00:00,4285.08,4371.52,3938.77,4108.37,1199.888264,NaN,NaN,NaN,False,False,0,False,False
2017-08-19 07:00:00,4108.37,4184.69,3850.00,4139.98,381.309763,NaN,NaN,NaN,False,False,0,False,False
2017-08-20 07:00:00,4120.98,4211.08,4032.62,4086.29,467.083022,NaN,NaN,NaN,False,False,0,False,False
2017-08-21 07:00:00,4069.13,4119.62,3911.79,4016.00,691.743060,NaN,NaN,NaN,False,False,0,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-03-08 07:00:00,37988.01,39362.08,37867.65,38730.63,55528.433670,-422.621053,-211.755972,-210.865081,False,False,0,False,False
2022-03-09 07:00:00,38730.63,42594.06,38656.45,41941.71,67392.587990,-218.615938,-6.200686,-212.415252,False,False,0,False,False
2022-03-10 07:00:00,41941.70,42039.63,38539.73,39422.00,71950.256770,-257.294029,-35.903021,-221.391008,False,False,0,False,False


In [10]:
df['action_price'] = df['Open'].shift(-1)
df.tail()

,Open,High,Low,Close,Volume,MACD_12_26_9,MACDh_12_26_9,MACDs_12_26_9,signal,TS_Trends,TS_Trades,TS_Entries,TS_Exits,action_price
Date,,,,,,,,,,,,,,
2022-03-08 07:00:00,37988.01,39362.08,37867.65,38730.63,55528.43367,-422.621053,-211.755972,-210.865081,False,False,0,False,False,38730.63
2022-03-09 07:00:00,38730.63,42594.06,38656.45,41941.71,67392.58799,-218.615938,-6.200686,-212.415252,False,False,0,False,False,41941.70
2022-03-10 07:00:00,41941.70,42039.63,38539.73,39422.00,71950.25677,-257.294029,-35.903021,-221.391008,False,False,0,False,False,39422.01
2022-03-11 07:00:00,39422.01,40236.26,38223.60,38729.57,59018.76420,-339.901832,-94.808659,-245.093173,False,False,0,False,False,38729.57
2022-03-12 07:00:00,38729.57,39327.99,38660.52,39184.99,3356.98064,-364.419719,-95.461237,-268.958482,False,False,0,False,False,NaN


In [11]:
trades_table = df.iloc[:,-5:][df.TS_Trades != 0]
trades_table['return'] = trades_table['action_price'].pct_change()
trades_table

,TS_Trends,TS_Trades,TS_Entries,TS_Exits,action_price,return
Date,,,,,,
2017-09-11 07:00:00,True,1,True,False,4208.60,NaN
2017-09-12 07:00:00,False,-1,False,True,4159.72,-0.011614
2017-09-30 07:00:00,True,1,True,False,4378.49,0.052592
2018-01-14 07:00:00,False,-1,False,True,13477.98,2.078226
2018-02-21 07:00:00,True,1,True,False,10439.02,-0.225476
2018-02-22 07:00:00,False,-1,False,True,9815.55,-0.059725
2018-02-27 07:00:00,True,1,True,False,10584.33,0.078323
2018-03-09 07:00:00,False,-1,False,True,9230.00,-0.127956
2018-04-19 07:00:00,True,1,True,False,8273.84,-0.103593


In [12]:
trades_summary = trades_table.loc[trades_table.TS_Exits == True]
trades_summary

,TS_Trends,TS_Trades,TS_Entries,TS_Exits,action_price,return
Date,,,,,,
2017-09-12 07:00:00,False,-1,False,True,4159.72,-0.011614
2018-01-14 07:00:00,False,-1,False,True,13477.98,2.078226
2018-02-22 07:00:00,False,-1,False,True,9815.55,-0.059725
2018-03-09 07:00:00,False,-1,False,True,9230.00,-0.127956
2018-05-16 07:00:00,False,-1,False,True,8330.00,0.006788
2018-08-07 07:00:00,False,-1,False,True,6720.63,-0.081560
2018-09-08 07:00:00,False,-1,False,True,6185.06,-0.120592
2018-10-11 07:00:00,False,-1,False,True,6252.71,-0.054805
2018-10-25 07:00:00,False,-1,False,True,6528.13,-0.031292


In [13]:
port = vbt.Portfolio.from_signals(df.Close,
                                  entries=signal_vectorbt.TS_Entries,
                                  exits=signal_vectorbt.TS_Exits,
                                  freq="D",
                                  init_cash=100000,
                                  fees=0.0025,
                                  slippage=0.0025)

In [14]:
port.plot().show()

In [15]:
port.stats()

Start                         2017-08-17 07:00:00
End                           2022-03-12 07:00:00
Period                         1669 days 00:00:00
Start Value                              100000.0
End Value                          1703905.624059
Total Return [%]                      1603.905624
Benchmark Return [%]                   814.451772
Max Gross Exposure [%]                      100.0
Total Fees Paid                      89863.773224
Max Drawdown [%]                        64.711144
Max Drawdown Duration           552 days 00:00:00
Total Trades                                   24
Total Closed Trades                            24
Total Open Trades                               0
Open Trade PnL                                0.0
Win Rate [%]                            33.333333
Best Trade [%]                         364.017833
Worst Trade [%]                        -16.562254
Avg Winning Trade [%]                  105.360039
Avg Losing Trade [%]                    -9.717107
